In [2]:
import warnings
warnings.filterwarnings(action='once')

import torch
import torch.nn.functional as F
import numpy as np
import json
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage.transform
import argparse
from scipy.misc import imread, imresize
from PIL import Image

import warnings
warnings.filterwarnings('ignore')

/home/pramod/.local/lib/python3.5/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
def read_mat_file(matfile):
    import scipy.io as sio
    
    mat_contents = sio.loadmat(matfile)
    data = mat_contents["data"]
    return data[0,0][7]

def load_matfile(img):
    if len(img.shape)==2:
        img = img[:,:,np.newaxis]
        img = np.concatenate([img,img,img],axis=2)
    
    img = imresize(img,(256,256))
    img = img.transpose(2, 0, 1)
    img = img /255
    img = torch.FloatTensor(img).to(device)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([normalize])
    image = transform(img)  # (3, 256, 256)

    return image

In [5]:
def load_image(image_path):
    # Read image and process
    img = imread(image_path)
    if len(img.shape) == 2:
        img = img[:, :, np.newaxis]
        img = np.concatenate([img, img, img], axis=2)
    img = imresize(img, (256, 256))
    img = img.transpose(2, 0, 1)
    img = img / 255.
    img = torch.FloatTensor(img).to(device)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([normalize])
    image = transform(img)  # (3, 256, 256)
    return image

In [6]:
def caption_image_beam_search(encoder, decoder, image_path, word_map, beam_size=3,read_mat = False):
    """
    Reads an image and captions it with beam search.

    :param encoder: encoder model
    :param decoder: decoder model
    :param image_path: path to image
    :param word_map: word map
    :param beam_size: number of sequences to consider at each decode-step
    :return: caption, weights for visualization
    """

    k = beam_size
    vocab_size = len(word_map)

    # Read image and process
    if read_mat:
        image = load_matfile(image_path)
    else:
        image = load_image(image_path)
    """    
    img = imread(image_path)
    if len(img.shape) == 2:
        img = img[:, :, np.newaxis]
        img = np.concatenate([img, img, img], axis=2)
    img = imresize(img, (256, 256))
    img = img.transpose(2, 0, 1)
    img = img / 255.
    img = torch.FloatTensor(img).to(device)
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    transform = transforms.Compose([normalize])
    image = transform(img)  # (3, 256, 256)
    """

    # Encode
    image = image.unsqueeze(0)  # (1, 3, 256, 256)
    encoder_out = encoder(image)  # (1, enc_image_size, enc_image_size, encoder_dim)
    enc_image_size = encoder_out.size(1)
    encoder_dim = encoder_out.size(3)

    # Flatten encoding
    encoder_out = encoder_out.view(1, -1, encoder_dim)  # (1, num_pixels, encoder_dim)
    num_pixels = encoder_out.size(1)

    # We'll treat the problem as having a batch size of k
    encoder_out = encoder_out.expand(k, num_pixels, encoder_dim)  # (k, num_pixels, encoder_dim)

    # Tensor to store top k previous words at each step; now they're just <start>
    k_prev_words = torch.LongTensor([[word_map['<start>']]] * k).to(device)  # (k, 1)

    # Tensor to store top k sequences; now they're just <start>
    seqs = k_prev_words  # (k, 1)

    # Tensor to store top k sequences' scores; now they're just 0
    top_k_scores = torch.zeros(k, 1).to(device)  # (k, 1)

    # Tensor to store top k sequences' alphas; now they're just 1s
    seqs_alpha = torch.ones(k, 1, enc_image_size, enc_image_size).to(device)  # (k, 1, enc_image_size, enc_image_size)

    # Lists to store completed sequences, their alphas and scores
    complete_seqs = list()
    complete_seqs_alpha = list()
    complete_seqs_scores = list()

    # Start decoding
    step = 1
    h, c = decoder.init_hidden_state(encoder_out)

    # s is a number less than or equal to k, because sequences are removed from this process once they hit <end>
    while True:

        embeddings = decoder.embedding(k_prev_words).squeeze(1)  # (s, embed_dim)

        awe, alpha = decoder.attention(encoder_out, h)  # (s, encoder_dim), (s, num_pixels)

        alpha = alpha.view(-1, enc_image_size, enc_image_size)  # (s, enc_image_size, enc_image_size)

        gate = decoder.sigmoid(decoder.f_beta(h))  # gating scalar, (s, encoder_dim)
        awe = gate * awe

        h, c = decoder.decode_step(torch.cat([embeddings, awe], dim=1), (h, c))  # (s, decoder_dim)

        scores = decoder.fc(h)  # (s, vocab_size)
        scores = F.log_softmax(scores, dim=1)

        # Add
        scores = top_k_scores.expand_as(scores) + scores  # (s, vocab_size)

        # For the first step, all k points will have the same scores (since same k previous words, h, c)
        if step == 1:
            top_k_scores, top_k_words = scores[0].topk(k, 0, True, True)  # (s)
        else:
            # Unroll and find top scores, and their unrolled indices
            top_k_scores, top_k_words = scores.view(-1).topk(k, 0, True, True)  # (s)

        # Convert unrolled indices to actual indices of scores
        prev_word_inds = top_k_words / vocab_size  # (s)
        next_word_inds = top_k_words % vocab_size  # (s)

        # Add new words to sequences, alphas
        seqs = torch.cat([seqs[prev_word_inds], next_word_inds.unsqueeze(1)], dim=1)  # (s, step+1)
        seqs_alpha = torch.cat([seqs_alpha[prev_word_inds], alpha[prev_word_inds].unsqueeze(1)],
                               dim=1)  # (s, step+1, enc_image_size, enc_image_size)

        # Which sequences are incomplete (didn't reach <end>)?
        incomplete_inds = [ind for ind, next_word in enumerate(next_word_inds) if
                           next_word != word_map['<end>']]
        complete_inds = list(set(range(len(next_word_inds))) - set(incomplete_inds))

        # Set aside complete sequences
        if len(complete_inds) > 0:
            complete_seqs.extend(seqs[complete_inds].tolist())
            complete_seqs_alpha.extend(seqs_alpha[complete_inds].tolist())
            complete_seqs_scores.extend(top_k_scores[complete_inds])
        k -= len(complete_inds)  # reduce beam length accordingly

        # Proceed with incomplete sequences
        if k == 0:
            break
        seqs = seqs[incomplete_inds]
        seqs_alpha = seqs_alpha[incomplete_inds]
        h = h[prev_word_inds[incomplete_inds]]
        c = c[prev_word_inds[incomplete_inds]]
        encoder_out = encoder_out[prev_word_inds[incomplete_inds]]
        top_k_scores = top_k_scores[incomplete_inds].unsqueeze(1)
        k_prev_words = next_word_inds[incomplete_inds].unsqueeze(1)

        # Break if things have been going on too long
        if step > 50:
            break
        step += 1
    #print(len(complete_seqs_scores))
    if len(complete_seqs_scores) == 0:
        seq = [9489]
        alphas = [0]
    else:
        i = complete_seqs_scores.index(max(complete_seqs_scores))
        seq = complete_seqs[i]
        alphas = complete_seqs_alpha[i]

    return seq, alphas

In [7]:
#warnings.filterwarnings('ignore')
# Load model
checkpoint = torch.load("models/BEST_checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar")
decoder = checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()
encoder = checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval()
print("Loaded encoder and decoder!")

Loaded encoder and decoder!


In [8]:
# Load word map (word2ix)
with open("models/WORDMAP_coco_5_cap_per_img_5_min_word_freq.json", 'r') as j:
    word_map = json.load(j)
rev_word_map = {v: k for k, v in word_map.items()}  # ix2word

In [10]:
images_test = "/home/pramod/Downloads/datasets/image_caption/pascal_image_data/test/"
image_eg = "image_1.jpg"
seq, alphas = caption_image_beam_search(encoder, decoder, images_test+image_eg, word_map, beam_size=5)

In [12]:
words = [rev_word_map[ind] for ind in seq]
print(words)
print(len(words))

['<start>', 'a', 'man', 'with', 'a', 'beard', 'is', 'holding', 'a', 'dog', '<end>']
11


In [13]:
import pandas as pd
import os

In [14]:
show_tell_attend_df  = pd.DataFrame()
count = 0

images = "/home/pramod/Downloads/datasets/image_caption/pascal_image_data/test/"
for img in os.listdir(images):
        if os.path.splitext(img)[1] == '.png' or os.path.splitext(img)[1] =='.jpeg' or os.path.splitext(img)[1] =='.jpg':
    
            seq, _ = caption_image_beam_search(encoder, decoder, images+img, word_map, beam_size=5)
            words = [rev_word_map[ind] for ind in seq]
            show_tell_attend_df=show_tell_attend_df.append(
            pd.DataFrame({"img":img,"show_attend_tell":str(words)},index=[0]),ignore_index=True)
            if count %500 ==0:
                print(img)
                print(seq)
                print(words)
            count +=1
            

image_1.jpg
[9488, 1, 2, 3, 1, 4070, 35, 55, 1, 974, 9489]
['<start>', 'a', 'man', 'with', 'a', 'beard', 'is', 'holding', 'a', 'dog', '<end>']
image_1449.jpg
[9488, 1, 570, 86, 17, 1, 68, 32, 1, 716, 9489]
['<start>', 'a', 'close', 'up', 'of', 'a', 'cow', 'in', 'a', 'field', '<end>']
image_19.jpg
[9488, 1, 974, 370, 6, 14, 587, 61, 23, 1, 974, 9489]
['<start>', 'a', 'dog', 'laying', 'on', 'the', 'ground', 'next', 'to', 'a', 'dog', '<end>']
image_2349.jpg
[9488, 1, 893, 123, 3, 1, 896, 28, 1, 2653, 9489]
['<start>', 'a', 'living', 'room', 'with', 'a', 'tv', 'and', 'a', 'fireplace', '<end>']
image_28.jpg
[9488, 1, 207, 799, 71, 61, 23, 1, 1975, 427, 9489]
['<start>', 'a', 'brown', 'horse', 'standing', 'next', 'to', 'a', 'wire', 'fence', '<end>']
image_3249.jpg
[9488, 1, 124, 98, 6, 1, 1603, 32, 1, 123, 9489]
['<start>', 'a', 'cat', 'sitting', 'on', 'a', 'bed', 'in', 'a', 'room', '<end>']
image_37.jpg
[9488, 1, 645, 409, 6, 14, 37, 17, 1, 289, 9489]
['<start>', 'a', 'car', 'parked', 'on',

In [15]:
matfile = '/home/pramod/Downloads/datasets/image_caption/labelme/data_for_image_caption.mat'

In [16]:
mat_data = False
if mat_data:
    #read matfile
    img_list = read_mat_file(matfile)
    print(len(img_list[0]))
    #perform caption generation
    show_tell_attend_df  = pd.DataFrame()
    count = 0

    for i in range(len(img_list[0])):
        seq, _ = caption_image_beam_search(encoder, decoder, img_list[0][i], word_map, beam_size=5,read_mat=False)
        words = [rev_word_map[ind] for ind in seq]
        show_tell_attend_df=show_tell_attend_df.append(
        pd.DataFrame({"img":i,"show_attend_tell":str(words)},index=[0]),ignore_index=True)
        if count %500 ==0:
            print(i)
            print(seq)
            print(words)
        count +=1

In [17]:
show_tell_attend_df.shape

(4952, 2)

In [18]:
show_tell_attend_df.to_csv("pascal_test.csv")

In [28]:
results = pd.read_csv("../show_tell/pascal_test.csv")
print(results.shape)

(4952, 3)


In [20]:
pd.merge(results,show_tell_attend_df,on="img").to_csv("pascal_test_show_attend_tell.csv")

# Calculate BLEU Score

In [21]:
from nltk.translate.bleu_score import sentence_bleu

In [29]:
show_tell_attend_df.head()

,img,show_attend_tell
0,image_1.jpg,"['<start>', 'a', 'man', 'with', 'a', 'beard', ..."
1,image_10.jpg,"['<start>', 'a', 'yellow', 'taxi', 'cab', 'dri..."
2,image_100.jpg,"['<start>', 'a', 'close', 'up', 'of', 'a', 'bi..."
3,image_1000.jpg,"['<start>', 'a', 'group', 'of', 'people', 'sta..."
4,image_1001.jpg,"['<start>', 'a', 'green', 'bus', 'parked', 'on..."


In [86]:
#reference - human translation
#candidate - machine translation

reference = show_tell_attend_df.iloc[1,1]
candidate = show_tell_attend_df.iloc[0,1]
score1 = sentence_bleu([reference], candidate)
score2 = sentence_bleu([candidate],reference)
print(score1, score2)

0.6223090127514708 0.6222160789161449


In [102]:
sentences_list = [show_tell_attend_df.iloc[i,1] for i in range(show_tell_attend_df.shape[0])]
sentence_scores = np.zeros((len(sentences_list),show_tell_attend_df.shape[0]))
sentence_scores.shape

(4952, 4952)

In [120]:
for idx,sentence in enumerate(sentences_list):
    for i in range(show_tell_attend_df.shape[0]):
        #print(idx,sentence)
        #print(show_tell_attend_df.iloc[i,1])
        sentence_scores[idx][i] = sentence_bleu([sentence],show_tell_attend_df.iloc[i,1])
        if i%4950 == 0 and idx%200==0:
            print(sentence_scores[idx][i],i,idx)
        

1.0 0 0
0.584908045268514 4950 0


KeyboardInterrupt: 

In [112]:
np.savetxt("sentence_scores_dataset.csv",sentence_scores,fmt='%.4f',delimiter=",")